In [3]:
import json
import random

def select_random_elements(json_data, num_elements):
    if num_elements >= len(json_data):
        return json_data
    return random.sample(json_data, num_elements)

def main():
    input_file='/home/qiang/projects/Digital_mental_health/final_dataset/SAS2UAS/Sorted_BLEU4_SAS2UAS.json'
    output_file='/home/qiang/projects/Digital_mental_health/final_dataset/experiments/datasets/SAS2UAS.json'

    with open(input_file, 'r', encoding='utf-8') as file:
        json_data = json.load(file)

    num_elements_to_select = 100
    selected_elements = select_random_elements(json_data, num_elements_to_select)

    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump(selected_elements, file, ensure_ascii=False, indent=4)

if __name__ == '__main__':
    main()


In [10]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)
text = "I'm doomed to live in a falling-apart house and drive an unreliable car. My life is a never-ending struggle, and I'll never be able to catch up."
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
# print(scores)
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


1) negative 0.9261
2) neutral 0.0662
3) positive 0.0076


In [29]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
from tqdm import tqdm
def get_sentiment_scores(text):
    MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    config = AutoConfig.from_pretrained(MODEL)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL)

    def preprocess(text):
        new_text = []
        for t in text.split(" "):
            t = '@user' if t.startswith('@') and len(t) > 1 else t
            t = 'http' if t.startswith('http') else t
            new_text.append(t)
        return " ".join(new_text)

    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    results = []
    for i in range(scores.shape[0]):
        label = config.id2label[ranking[i]]
        score = scores[ranking[i]]
        results.append((label, np.round(float(score), 4)))
    sentiment_dict = {label: score for label, score in results}
    return sentiment_dict

def process_json_file(input_file, output_file):
    with open(input_file, 'r') as f:
        data = json.load(f)
    
    for item in tqdm(data):
        post = item['Post']
        transferred_post = item['Transferred_Post']
        post_sentiment_scores = get_sentiment_scores(post)
        transferred_post_sentiment_scores = get_sentiment_scores(transferred_post)
        item['Post_Postive'] = post_sentiment_scores['positive']
        item['Post_Negative'] = post_sentiment_scores['negative']
        item['Post_Neutral'] = post_sentiment_scores['neutral']
        item['Transferred_Post_Postive'] = transferred_post_sentiment_scores['positive']
        item['Transferred_Post_Negative'] = transferred_post_sentiment_scores['negative']
        item['Transferred_Post_Neutral'] = transferred_post_sentiment_scores['neutral']

    with open(output_file, 'w') as f:
        json.dump(data, f, indent=4)

input_file = "/home/qiang/projects/Digital_mental_health/final_dataset/experiments/datasets/SAS2UAS.json"
output_file = "/home/qiang/projects/Digital_mental_health/final_dataset/experiments/datasets/SAS2UAS_sentiment.json"
process_json_file(input_file, output_file)


  0%|          | 0/100 [00:00<?, ?it/s]Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are 

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt

def calculate_average(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    post_positive = []
    post_negative = []
    post_neutral = []
    transferred_post_positive = []
    transferred_post_negative = []
    transferred_post_neutral = []

    for item in data:
        post_positive.append(item['Post_Postive'])
        post_negative.append(item['Post_Negative'])
        post_neutral.append(item['Post_Neutral'])
        transferred_post_positive.append(item['Transferred_Post_Postive'])
        transferred_post_negative.append(item['Transferred_Post_Negative'])
        transferred_post_neutral.append(item['Transferred_Post_Neutral'])

    post_positive_avg = np.mean(post_positive)
    post_negative_avg = np.mean(post_negative)
    post_neutral_avg = np.mean(post_neutral)
    transferred_post_positive_avg = np.mean(transferred_post_positive)
    transferred_post_negative_avg = np.mean(transferred_post_negative)
    transferred_post_neutral_avg = np.mean(transferred_post_neutral)

    return (post_positive_avg, post_negative_avg, post_neutral_avg, transferred_post_positive_avg, transferred_post_negative_avg, transferred_post_neutral_avg)

def plot_bar_chart(averages):
    labels = ['Post_Positive', 'Post_Negative', 'Post_Neutral', 'Transferred_Post_Positive', 'Transferred_Post_Negative', 'Transferred_Post_Neutral']
    values = averages

    fig, ax = plt.subplots()
    ax.bar(labels[:3], values[:3], color='blue', label='Post')
    ax.bar(labels[3:], values[3:], color='orange', label='Transferred_Post')
    ax.set_ylabel('Average Score')
    ax.set_title('Average Sentiment Scores')
    ax.legend()
    plt.show()

json_file = "/home/qiang/projects/Digital_mental_health/final_dataset/experiments/datasets/GAS2SPAS_sentiment.json"
averages = calculate_average(json_file)
print("Average Scores:")
print("Post Positive:", averages[0])
print("Post Negative:", averages[1])
print("Post Neutral:", averages[2])
print("Transferred Post Positive:", averages[3])
print("Transferred Post Negative:", averages[4])
print("Transferred Post Neutral:", averages[5])

plot_bar_chart(averages)
